In [28]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import nltk
import re

In [3]:
data = pd.read_csv('data.csv')

C:\Users\1\AppData\Local\Temp\ipykernel_6204\2948258669.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data.csv')


In [4]:
data.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [5]:
data.shape

(800975, 6)

# Отбор 500 значений в каждой из 5 категорий

In [14]:
topics = ['Путешествия', 'Ценности', 'Мир', 'Наука и техника', 'Экономика']
news_in_cat_count = 500

In [19]:
df_res = pd.DataFrame()
for topic in topics:
    df_topic = data[data['topic'] == topic][:news_in_cat_count]
    df_res = df_res.append(df_topic, ignore_index=True)

C:\Users\1\AppData\Local\Temp\ipykernel_6204\2932356076.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_res = df_res.append(df_topic, ignore_index=True)


# Функция, обрабатывающая текст

In [32]:
texts = df_res['text']

In [36]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# Стоп-слова

In [88]:
stopwords = nltk.corpus.stopwords.words('russian')
#можно расширить список стоп-слов
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])

# TF-IDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_featur=200000
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=stopwords,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3))

In [38]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

D:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn(


CPU times: total: 26 s
Wall time: 26.2 s


In [42]:
type(tfidf_matrix)
tfidf_matrix

<2500x2182 sparse matrix of type '<class 'numpy.float64'>'
	with 187085 stored elements in Compressed Sparse Row format>

# Kmeans clustering

In [45]:
num_clusters = 5

from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters, random_state=777)

In [46]:
%%time
km.fit(tfidf_matrix)

CPU times: total: 5.7 s
Wall time: 2.07 s


KMeans(n_clusters=5, random_state=777)

In [47]:
%%time
idx = km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: total: 5.66 s
Wall time: 1.97 s


In [53]:
clusterkm = km.labels_.tolist()
frame = pd.DataFrame(texts)

#k-means
out = { 'text': texts, 'cluster': clusterkm, 'topic': df_res['topic'] }
frame1 = pd.DataFrame(out, columns = ['text', 'cluster', 'topic'])

# Просмотр того, как распределились кластеры

In [54]:
frame1.head(10)

,text,cluster,topic
0,"Названы самые раздражающие фотографии, которые...",3,Путешествия
1,Австрийская авиакомпания FlyNiki открывает 10-...,3,Путешествия
2,Премьер-министр Греции Алексис Ципрас призвал ...,3,Путешествия
3,Китайский лоукостер Spring Airlines собрался п...,3,Путешествия
4,Большинство российских путешественников отметя...,3,Путешествия
5,Китай планирует создать международную туристич...,3,Путешествия
6,Туристический сервис Tripadvisor определил луч...,3,Путешествия
7,"Число российских туристов, выбравших в качеств...",3,Путешествия
8,Авиакомпания Iraqi Airways в ближайшее время н...,3,Путешествия
9,"Посол Палестины в России Фаед Мустафа сообщил,...",3,Путешествия


In [77]:
for i in range(0, 5):
    print(i)
    print(frame1[frame1['cluster'] == i]['topic'].value_counts())
    print("------------------------------------------------------")

0
Наука и техника    162
Мир                 10
Name: topic, dtype: int64
------------------------------------------------------
1
Экономика          443
Мир                 53
Путешествия          7
Наука и техника      6
Ценности             3
Name: topic, dtype: int64
------------------------------------------------------
2
Ценности           394
Наука и техника      1
Name: topic, dtype: int64
------------------------------------------------------
3
Путешествия        409
Ценности             9
Экономика            3
Мир                  1
Наука и техника      1
Name: topic, dtype: int64
------------------------------------------------------
4
Мир                436
Наука и техника    330
Ценности            94
Путешествия         84
Экономика           54
Name: topic, dtype: int64
------------------------------------------------------


### Видно, что кластеры распределились относительно равномерно, кроме 0 и особенно 4, но при увеличении данных или смене модели, смене гиперпараметров можно будет улучшить качество результата

# --------------------------------------------------------------------------------------------------

# Обработка для классификации

In [123]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])


def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])


import re
def remove_multiple_spaces(text):
    return re.sub(r'\s+', ' ', text, flags=re.I)


def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)


from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])

In [110]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in df_res['text']]

In [112]:
df_res['text_prep'] = prep_text

# Лемматизация

In [115]:
lemm_texts_list = []
for text in tqdm(df_res['text_prep']):
    #print(text)
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df_res['text_lemm'] = lemm_texts_list

  0%|          | 0/2500 [00:00<?, ?it/s]

# Стемматизация

In [118]:
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian") 

text = df_res['text_prep'][0]
word_tokenize(text) 

from nltk import word_tokenize

stemmed_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_res['text_stem'] = stemmed_texts_list

  0%|          | 0/2500 [00:00<?, ?it/s]

## Разница в потраченном времени просто колоссальна, так у лемматизации получение текста происходило с 1 операцией/ 1 сек, то стемматизация 154 оп/ 1 сек

In [124]:
df_res.head()

,url,title,text,topic,tags,date,text_prep,text_lemm,text_stem
0,https://lenta.ru/news/2014/06/26/annoyingphotos/,Составлен рейтинг самых раздражающих отпускных...,"Названы самые раздражающие фотографии, которые...",Путешествия,Мнения,2014/06/26,названы самые раздражающие фотографии которые ...,называть самый раздражающий фотография который...,назва сам раздража фотограф котор уеха отпуск ...
1,https://lenta.ru/news/2015/01/09/flight/,В Европе появится 10-минутный авиарейс,Австрийская авиакомпания FlyNiki открывает 10-...,Путешествия,Мир,2015/01/09,австрийская авиакомпания flyniki открывает мин...,австрийский авиакомпания flyniki открывать мин...,австрийск авиакомпан flyniki открыва минутн ав...
2,https://lenta.ru/news/2015/01/28/tsipras/,Греческий премьер призвал избавиться от отелей...,Премьер-министр Греции Алексис Ципрас призвал ...,Путешествия,Мир,2015/01/28,премьер министр греции алексис ципрас призвал ...,премьер министр греция алексис ципрас призыват...,премьер министр грец алексис ципрас призва отк...
3,https://lenta.ru/news/2015/02/03/standfly/,Китайская авиакомпания собралась ввести в само...,Китайский лоукостер Spring Airlines собрался п...,Путешествия,Мир,2015/02/03,китайский лоукостер spring airlines собрался п...,китайский лоукостер spring airlines собираться...,китайск лоукостер spring airlines собра предло...
4,https://lenta.ru/news/2015/02/12/tripforvalent...,Россияне отправятся отмечать День святого Вале...,Большинство российских путешественников отметя...,Путешествия,Россия,2015/02/12,большинство российских путешественников отметя...,большинство российский путешественник отмечать...,большинств российск путешественник отмет ден с...


# Деление данных

In [136]:
X = df_res['text_lemm']
y = df_res['topic']

In [137]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [149]:
my_tags = df_res['topic'].unique()
my_tags

array(['Путешествия', 'Ценности', 'Мир', 'Наука и техника', 'Экономика'],
      dtype=object)

# Naive Bayes Classifier

In [126]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [127]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [138]:
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

## Метрики

In [139]:
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

In [150]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.8986666666666666
                 precision    recall  f1-score   support

    Путешествия       0.94      0.69      0.80       154
       Ценности       0.91      0.91      0.91       165
            Мир       0.93      0.97      0.95       158
Наука и техника       0.93      0.97      0.95       138
      Экономика       0.80      0.96      0.88       135

       accuracy                           0.90       750
      macro avg       0.90      0.90      0.90       750
   weighted avg       0.90      0.90      0.90       750



# Logistic Regression

In [155]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, max_iter=200)),
               ])

In [156]:
%%time
logreg.fit(X_train, y_train)

CPU times: total: 2.39 s
Wall time: 2.34 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(C=100000.0, max_iter=200, n_jobs=1))])

In [157]:
%%time
y_pred = logreg.predict(X_test)

CPU times: total: 78.1 ms
Wall time: 89.8 ms


In [158]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.924
                 precision    recall  f1-score   support

    Путешествия       0.90      0.79      0.84       154
       Ценности       0.90      0.95      0.92       165
            Мир       0.99      0.98      0.99       158
Наука и техника       0.97      1.00      0.98       138
      Экономика       0.86      0.90      0.88       135

       accuracy                           0.92       750
      macro avg       0.92      0.92      0.92       750
   weighted avg       0.92      0.92      0.92       750

